In [1]:
#!pip install learn2learn
#!pip install torchmeta
import torch
import utils
import model as MODEL
import train as TRAIN

from google.colab import drive
drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__ == '__main__':
    Params = {'datasize':96,'nways':32,'kshots':1,'in_channels':3,
              'num_classes':45,'hidden_size':2, "innerStep":.005,
              'MetaLR':.01,"number_of_tasks":1, "Order":True,"outerVSinner": 1,"epoch":50,'aug':True}
    # print('Getting data')
    train_tasks,val_loader,test_loader,Params=utils.colabgetdata(Params)

   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/utils.py:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xtest = torch.tensor(test_data).permute(0, 3, 1, 2)


In [2]:
model = MODEL.ConvolutionalNeuralNetwork(Params)
model = model.to(device)
loss_rate=0
for i in range(50):
    TRAIN.train(model,train_tasks,Params,val_loader)
    loss_rate = utils.getvalErr(model,val_loader)[0]
print(loss_rate)




0.5482954382896423


In [ ]:
reduce = [True,False]
nwayss =[32,20,10,5]
for i in reduce:
    Params['Order']=i
    for tasknum in range(4): ## number of tasks inside each outer loop
        Params["number_of_tasks"] = int(2**tasknum)
        for InoutRatio in range(2): ## ratio of data in outer loop of task to one of the data in tasks right now it is 1:1, 2:1, 3:1
            Params['outerVSinner'] = InoutRatio+1
            for meta in range(4):## learning rates
                Params['MetaLR'] *=(1**(-meta+1))
                for innLR in range(4):
                    Params["innerStep"] *=(1**(-innLR+1))
                    for nwayys in nwayss:
                        Params['nways']=nwayys
                    model = MODEL.ConvolutionalNeuralNetwork(Params)
                    model = model.to(device)
                    Atrain = 0
                    Ltrain  =0
                    Aval = 0
                    Akval = 0
                    for i in range(Params['epoch']):
                        Atrain, Ltrain = TRAIN.train(model, train_tasks,Params, val_loader)
                        Aval, Akval = utils.getvalErr(model, val_loader)
                        if ((i <= 3) and Aval < .1): ## not to waste time stop early
                             break
                        if Aval > loss_rate: ## if our val is better then what we had save it
                            print("found one better")
                            print(loss_rate)
                            print(Params)
                            dict = {'Training loss': Ltrain, "Training Error Rate": Atrain, "Validation topk": Akval,
                                    "Validation Error Rate": Aval}
                            tofile = {**dict, **Params}
                            with open('/content/drive/MyDrive/Project/BestSoFar.txt', 'w') as f:
                                print(tofile, file=f)
                                loss_rate = dict["Validation Error Rate"]